In [1]:
# importing libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import os
import math

In [2]:
# importing elpileptic and healthy data
def load_csv_files(directory):
    all_df = []    
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            all_df.append(df)
    
    combined_df = pd.concat(all_df, ignore_index=True)   
    return combined_df

epi_directory = '/kaggle/input/epileptic'
healthy_directory = '/kaggle/input/healthy'

# loading epileptical data 
epi_data = load_csv_files(epi_directory)
# loading healthy data
heal_data = load_csv_files(healthy_directory)

In [ ]:
# converting the dataframe into 2-D numpy-array
epi_data = np.asarray(epi_data.T)
heal_data = np.asarray(heal_data.T)
# adjusting sizes
epi_resample_data = np.zeros_like(heal_data)
for i in np.arange(0, 19):
    epi_resample_data[i] = epi_data[i][ : 1207161]

In [4]:
epi_resample_data.shape, heal_data.shape

((19, 1207161), (19, 1207161))

### Second Feautre : Approximate Entropy

In [5]:
def approximate_entropy(time_series, m, r):
    """  
    Parameters:
    - time_series: The time series data (a list or numpy array).
    - m: Length of compared run of data (usually a small integer).
    - r: Filtering level (a positive number, often 0.2 * std of the time series).
    Returns:
    - ApEn value.
    """  
    def _phi(m):
        N = len(time_series)
        x = [time_series[i : i + m] for i in range(N - m + 1)]
        C = [np.sum(np.max(np.abs(x - x[j]), axis=1) <= r) / (N - m + 1.0) for j in range(len(x))]
        return np.sum(np.log(C)) / (N - m + 1.0) 
    return _phi(m) - _phi(m + 1) 

def break_time_series(time_series, length=250):
    # Calculate the length of padding needed
    padding_needed = length - (len(time_series) % length)
    
    # If padding is needed, calculate the mean of the time series
    if padding_needed > 0:
        mean_value = np.mean(time_series)
        padding = [mean_value] * padding_needed
        time_series = np.concatenate([time_series, padding])
    
    # Break the time series into equal intervals of 50
    num_intervals = len(time_series) // length
    broken_series = np.array_split(time_series, num_intervals)  
    #features::
    #interval_means     =    [np.mean(interval) for interval in broken_series]
    #svd_features       =    [np.mean(interval) for interval in broken_series]
    entropy_feature    =    [approximate_entropy(interval,1000,0.2 * np.std(interval)) for interval in broken_series]  
    return entropy_feature   

#Feature extractor for all channels:
def feature_extractor_concat(data, length):
    feature=[]
    for i in range(0,19):
        m=break_time_series(pd.DataFrame(data).T.iloc[i], length)
        feature.append(m)
    feature=pd.DataFrame(feature)
    return feature

In [ ]:
# calculating approx_entropy for a all channels
length = heal_data.shape[1]
epi_appEntropy_feautre = feature_extractor_concat(epi_resample_data, length)
healt_appEntropy_feautre = feature_extractor_concat(heal_data, length)

In [ ]:
# epi_appEntropy_feautres = []
# healt_appEntropy_feautres = []
# length = heal_data.shape[1]

# for i in range(0, 2):
#     epi_appEntropy_feautre = feature_extractor_concat(epi_resample_data[i], length)
#     healt_appEntropy_feautre = feature_extractor_concat(heal_data[i], length)
#     epi_appEntropy_feautres.append(epi_appEntropy_feautre)
#     healt_appEntropy_feautres.append(healt_appEntropy_feautre)
    
# epi_appEntropy_feautres = np.asarray(epi_appEntropy_feautres)
# healt_appEntropy_feautres = np.asarray(healt_appEntropy_feautres)